In [ ]:
#How many stores does business have in which countries?

SELECT country_code, count(*) as occurances
FROM dim_store_details
GROUP BY country_code
order by occurances desc;

![](_screenshot1.png)


In [ ]:
#Which location has most stores?

SELECT locality, count(*) as occurances
FROM dim_store_details
GROUP BY locality
order by occurances desc;

In [ ]:
#Which months has highest sales

SELECT month,
SUM(product_quantity*product_price) as total_price
from orders_table
JOIN dim_products on orders_table.product_code = dim_products.product_code
JOIN dim_date_times on orders_table.date_uuid = dim_date_times.date_uuid
GROUP by month
ORDER by total_price desc;

In [ ]:
# How many sales were coming from Online?
SELECT SUM(product_quantity) AS total_product_quantity, store_code
FROM orders_table
WHERE store_code = 'WEB-1388012W'
GROUP BY store_code

UNION

SELECT SUM(product_quantity) AS total_product_quantity, 'Others' as store_code
FROM orders_table
WHERE store_code <> 'WEB-1388012W'


In [ ]:
#What percentage of sales comes from each store?
SELECT store_type,
SUM(product_quantity*product_price) as total_price,
ROUND(count(*) * 100.0 / (select count(*) from orders_table),2) as percentage

from orders_table
JOIN dim_products on orders_table.product_code = dim_products.product_code
JOIN dim_store_details on orders_table.store_code = dim_store_details.store_code
GROUP BY store_type
ORDER by total_price desc;

In [ ]:
#Which month in each year produced the highest cost of sales?

WITH ranked_prices AS (
  SELECT
    year,
    month,
    SUM(product_quantity * product_price) AS total_price,
    RANK() OVER (PARTITION BY year ORDER BY SUM(product_quantity * product_price) DESC) AS price_rank
  FROM orders_table
  JOIN dim_products ON orders_table.product_code = dim_products.product_code
  JOIN dim_date_times ON orders_table.date_uuid = dim_date_times.date_uuid
  GROUP BY year, month
)
SELECT year, month, total_price
FROM ranked_prices
WHERE price_rank = 1
ORDER BY year, month;


SELECT year,month,
SUM(product_quantity*product_price) as total_price
from orders_table
JOIN dim_products on orders_table.product_code = dim_products.product_code
JOIN dim_date_times on orders_table.date_uuid = dim_date_times.date_uuid
GROUP by year, month
ORDER by total_price desc;

In [ ]:
#What is our staff headcount?

SELECT sum(staff_numbers) as total_staffs, country_code
FROM dim_store_details
GROUP by country_code
order by total_staffs desc;

In [ ]:
#Which German store type is selling most?
SELECT store_type, 
SUM(product_quantity*product_price) as total_price,
country_code 
FROM orders_table
JOIN dim_products on orders_table.product_code = dim_products.product_code
JOIN dim_store_details on orders_table.store_code = dim_store_details.store_code
WHERE country_code = 'DE'
GROUP by store_type, country_code
order by total_price asc;



In [ ]:
#How quickly is the company making sales?

WITH cte AS 
(
  SELECT  
    year,
    TO_TIMESTAMP(CONCAT_WS('-', year, month, day, timestamp), 'YYYY-MM-DD-HH24:MI:SS') as sales_time
  FROM 
    dim_date_times
  ORDER BY 
    year, sales_time ASC
),
next_sales_duration AS 
(
  SELECT 
    year,
    LEAD(sales_time) OVER (PARTITION BY year ORDER BY sales_time ASC) - sales_time AS duration
  FROM 
    cte
)
 
SELECT 
  year, 
  CONCAT(
    '{"hours": ', FLOOR(AVG(EXTRAC
    '{"hours": ', FLOOR(AVG(EXTRACT(EPOCH FROM next_sales_duration.duration)) / 3600),
    ', "minutes": ', FLOOR((AVG(EXTRACT(EPOCH FROM next_sales_duration.duration)) % 3600) / 60),
    ', "seconds": ', FLOOR(AVG(EXTRACT(EPOCH FROM next_sales_duration.duration)) % 60),
    ' }') AS avg_sales_duration,
  AVG(EXTRACT(EPOCH FROM duration)) as avg_sales_duration_seconds

FROM 
  next_sales_duration
GROUP BY 
  year
ORDER BY 
  avg_sales_duration_seconds desc;